# Load Data

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', 500)

In [2]:
def load_df(csv_path='../data/raw/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
train_filepath = '../data/raw/train.csv'
test_filepath = '../data/raw/test.csv'

In [4]:
%%time
df_train = load_df(csv_path=train_filepath)

Loaded train.csv. Shape: (903653, 55)
Wall time: 4min 54s


In [5]:
# df_train = pd.read_csv(train_filepath)
# df_train = pd.read_csv(test_filepath)

# Describe data

In [6]:
df_train.shape

(903653, 55)

In [7]:
df_train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Izmir,not available in demo dataset,Asia,Turkey,not available in demo dataset,not available in demo dataset,(not set),ttnet.com.tr,not available in demo dataset,Izmir,Western Asia,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Oceania,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,Australasia,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,Madrid,not available in demo dataset,Europe,Spain,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Community of Madrid,Southern Europe,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not a

In [8]:
df_train.dtypes

channelGrouping                                      object
date                                                  int64
fullVisitorId                                        object
sessionId                                            object
socialEngagementType                                 object
visitId                                               int64
visitNumber                                           int64
visitStartTime                                        int64
device.browser                                       object
device.browserSize                                   object
device.browserVersion                                object
device.deviceCategory                                object
device.flashVersion                                  object
device.isMobile                                        bool
device.language                                      object
device.mobileDeviceBranding                          object
device.mobileDeviceInfo                 

# Data cleaning

In [53]:
# Convert total columns to float
float_columns = {}
for col in df_train.columns:
    if 'total' in col:
        float_columns[col] = float
        
df_train = df_train.astype(float_columns)

In [21]:
# Get number of unique and nulls for each column, add to delete list if 1 unique and 0 nulls
columns_to_drop = []
for col in df_train.columns:
    print(col)
    _nunique = df_train[col].nunique()
    _nnulls = df_train[col].isnull().sum()
    print('Unique: ',_nunique)
    print('Nulls: ',_nnulls)
    if (_nunique == 1) & (_nnulls == 0):
        columns_to_drop.append(col)
        print('Added to columns to drop')
    print('-' * 30)

channelGrouping
Unique:  8
Nulls:  0
------------------------------
date
Unique:  366
Nulls:  0
------------------------------
fullVisitorId
Unique:  714167
Nulls:  0
------------------------------
sessionId
Unique:  902755
Nulls:  0
------------------------------
visitId
Unique:  886303
Nulls:  0
------------------------------
visitNumber
Unique:  384
Nulls:  0
------------------------------
visitStartTime
Unique:  887159
Nulls:  0
------------------------------
device.browser
Unique:  54
Nulls:  0
------------------------------
device.deviceCategory
Unique:  3
Nulls:  0
------------------------------
device.isMobile
Unique:  2
Nulls:  0
------------------------------
device.operatingSystem
Unique:  20
Nulls:  0
------------------------------
geoNetwork.city
Unique:  649
Nulls:  0
------------------------------
geoNetwork.cityId
Unique:  1
Nulls:  0
Added to columns to drop
------------------------------
geoNetwork.continent
Unique:  6
Nulls:  0
------------------------------
geoNetwo

In [25]:
# Drop columns from columns_to_drop list
df_train.drop(columns_to_drop, axis=1, inplace=True)

In [37]:
# Show unique values of columns with null values
null_columns = df_train.columns[df_train.isnull().sum() > 0]
for col in null_columns:
    print(col)
    print(df_train[col].unique())
    print('-' * 30)

totals.bounces
['1' nan]
------------------------------
totals.newVisits
['1' nan]
------------------------------
totals.pageviews
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '20' '19' '21' '25' '22' '26' '23' '24' '30' '27' '29' '28'
 '32' '31' '34' '33' '38' '35' '40' '36' '37' '39' '43' '45' '47' '52'
 '44' '50' '46' '49' '51' '56' '59' '71' '57' '82' '84' '92' '90' '41'
 '79' '138' '48' '69' '112' '42' '65' '60' '63' '68' '58' '74' '165' '53'
 '54' '55' '61' '70' '80' '78' '72' '188' '73' '62' '75' '86' '197' '148'
 '67' '102' '64' '103' '144' nan '81' '119' '104' '128' '87' '111' '66'
 '83' '135' '94' '91' '99' '77' '118' '202' '127' '93' '106' '155' '117'
 '76' '89' '85' '143' '105' '115' '95' '110' '137' '120' '116' '208' '126'
 '400' '101' '108' '333' '97' '96' '98' '125' '113' '327' '154' '121'
 '183' '123' '141' '224' '88' '195' '343' '156' '193' '100' '150' '122'
 '145' '174' '147' '131' '169' '164' '249' '114' '189' '223' '124' '139'
 

<pre>
Next steps:
- Fill NA values
- Plot correlations, hist, scatter, violin, etc